### Laboratorio 6 - Ejercicio 4
Stefano Aragoni, Carol Arévalo, Luis Santos, Diego Perdomo

-----------

`Ejercicio 4`

Tiene la tarea de <font color=orange>modelar el flujo de tráfico urbano a múltiples escalas, desde vehículos individuales hasta toda la ciudad</font>. Implemente un modelo de flujo de tráfico multiescala en Python para analizar el comportamiento del tráfico y los patrones de congestión.


##### Dependencias

Como primer paso, se importan las librerías necesarias para el desarrollo del ejercicio.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from IPython.display import display, Image as IPImage

##### Funciones y clases

Se definen las funciones y clases necesarias para el desarrollo del ejercicio.

En este caso, el modelo individual es representado por la clase de Vehicle. El tipo puede ser sedan, van, bicicleta, o un bus.

Asimismo, se modela la carretera, y las intersecciones con semáforos.

In [2]:
speeds = {
    'sedan': 45,
    'van': 37.5,
    'bike': 55,
    'bus': 40
}

class Vehicle:
    def __init__(self, i_speed, a, pos, lane, tipo):
        self.speed = i_speed
        self.acceleration = a
        self.pos = pos
        self.lane = lane
        self.tipo = tipo


    # formula de aceleracion: v^2 = v0^2 + 2*a*d
    def frenar_acelerar(self, d, max_speed, semaforo):
        #print('  - speed: ', self.speed)
        #print('  - d: ', d)
        #print('  - pos: ', self.pos)
        #print('  - semaforo: ', semaforo.pos)
        #print('  - semaforo state: ', semaforo.state)
        
        #if self.pos in range(int(), semaforo.pos) and self.pos < semaforo.pos:    
        #print('  - semaforo: ',  self.pos, '(', semaforo.pos - (speeds[self.tipo] / 3.6 * 5)  ,semaforo.pos , ')' , semaforo.state)
        if self.pos >= semaforo.pos - (speeds[self.tipo] / 3.6) *5 and self.pos <= semaforo.pos:
            #print('  - flag 1: ', self.pos, self.speed)
            if semaforo.state == 'red':
                #print('  - frenar')
                self.speed = 0
            else:
                #print('  - acelerar')
                if self.speed == 0: # si paró en el semaforo, acelera de golpe
                    #self.speed = speeds[self.tipo] / 3.6 - 2*self.acceleration*5
                    temp = speeds[self.tipo] / 3.6
                    temp = temp**2
                    #print('  - temp: ', temp)
                    temp2 = 2*self.acceleration*5
                    #print('  - temp2: ', temp2)
                    self.speed = math.sqrt(temp - temp2)
                else:
                    if self.speed < max_speed:
                        if d > 5: # si la distancia es mayor a 10 metros, acelera
                            #print('  - flag 2: ', self.pos, self.speed)
                            temp = self.speed**2 + 2*self.acceleration*d
                            self.speed = math.sqrt(temp)
                        #self.speed = math.sqrt(self.speed**2 + 2*self.acceleration*d) 
                        elif d < 5 and d > 0:
                            #print('  - flag 3: ', self.pos, self.speed)
                            temp = self.speed**2 - 2*self.acceleration*d
                            if temp >= 0:
                                self.speed = math.sqrt(temp)
                            else:
                                self.speed = 0
                        elif d <= 0:
                            #print('  - flag 4: ', self.pos, self.speed)
                            # Frenar de golpe
                            self.speed = 0
                    else:
                        if d < 5 and d > 0:
                            #print('  - flag 5: ', self.pos, self.speed)
                            temp = self.speed**2 - 2*self.acceleration*d
                            if temp >= 0:
                                self.speed = math.sqrt(temp)
                            else:
                                self.speed = 0
                        elif d <= 0:
                            #print('  - flag 6: ', self.pos, self.speed)
                            # Frenar de golpe
                            self.speed = 0
        else:
            if self.speed < max_speed:
                if d > 5: # si la distancia es mayor a 10 metros, acelera
                    #print('  - flag 2: ', self.pos, self.speed)
                    temp = self.speed**2 + 2*self.acceleration*d
                    self.speed = math.sqrt(temp)
                #self.speed = math.sqrt(self.speed**2 + 2*self.acceleration*d) 
                elif d < 5 and d > 0:
                    #print('  - flag 3: ', self.pos, self.speed)
                    temp = self.speed**2 - 2*self.acceleration*d
                    if temp >= 0:
                        self.speed = math.sqrt(temp)
                    else:
                        self.speed = 0
                elif d <= 0:
                    #print('  - flag 4: ', self.pos, self.speed)
                    # Frenar de golpe
                    self.speed = 0
            else:
                if d < 5 and d > 0:
                    #print('  - flag 5: ', self.pos, self.speed)
                    temp = self.speed**2 - 2*self.acceleration*d
                    if temp >= 0:
                        self.speed = math.sqrt(temp)
                    else:
                        self.speed = 0
                elif d <= 0:
                    #print('  - flag 6: ', self.pos, self.speed)
                    # Frenar de golpe
                    self.speed = 0
        #print('-'*50)


class Road:
    def __init__(self, length, speed_limit, lanes=2):
        self.length = length
        self.speed_limit = speed_limit
        self.lanes = lanes


# Semaforo binario: sólo existe luz verde y roja, no amarilla
class Semaforo:
    def __init__(self, pos, state, red, green):
        self.pos = pos
        self.state = state
        self.red = red
        self.green = green
        self.timer = 0

    def semaforo_update(self, dt):
        if self.state == 'green':
            self.timer += dt
            if self.timer == self.green:
                self.state = 'red'
                self.timer = 0
        elif self.state == 'red':
            self.timer += dt
            if self.timer == self.red:
                self.state = 'green'
                self.timer = 0



##### Simulación del tráfico

Posteriormente, se define la simulación del tráfico. Se define el número de vehículos, el tiempo de simulación, y la velocidad máxima de los vehículos.

In [3]:

def simulate_traffic(road, vehicles, semaforos, duration, dt):

    duration_s = duration * 3600 # convertir horas a segundos

    flow_speeds = []

    i = 0
    while i < duration_s and len(vehicles) > 0:
        #for v in vehicles:
        #    print(v.pos, '  -----  ', v.speed)
        #print('-'*50)


        i += dt
        for s in semaforos:
            s.semaforo_update(dt)
        
        for v in vehicles:
            
            distance_to_next = road.length
            next_semaforo = None
            # distancia al siguiente vehiculo o semaforo
            next_vehicle_distances = [abs(j.pos - v.pos) for j in vehicles if j != v and j.lane == v.lane]
            next_semaforo_distances = [abs(j.pos - v.pos) for j in semaforos]
            next_v_d = min(next_vehicle_distances) if next_vehicle_distances else road.length
            next_s_d = min(next_semaforo_distances) if next_semaforo_distances else road.length
            next_semaforo = semaforos[next_semaforo_distances.index(next_s_d)] 
            #quemascerca = "vehiculo" if next_v_d < next_s_d else "semaforo"
            #print('quemascerca: ', quemascerca)
            distance_to_next = min(next_v_d, next_s_d)
            v.frenar_acelerar(distance_to_next, road.speed_limit, next_semaforo)
            if (v.pos + v.speed * dt / 3.6) < road.length:
                v.pos += v.speed * dt / 3.6
            else:
                # Si el vehiculo llega al final de la carretera, se elimina del arreglo
                vehicles.remove(v)
        
        if len(vehicles) > 0:
            flow_speed = sum([v.speed for v in vehicles]) / len(vehicles)
            flow_speeds.append((i, flow_speed))        
    
    
        
    
    return flow_speeds




# conversión de km/h a m/s: 1 km/h * 1000 m/km * 1 h/3600 s = 1/3.6 m/s
# conversión de km/h^2 a m/s^2: 1 km/h^2 * 1000 m/km * 1 h/3600 s * 1 h/3600 s = 1000/(3600^2) m/s^2

##### Visualización de la simulación

Finalmente, utilizando la función de simulación de tráfico, se visualiza el comportamiento de los vehículos en la carretera. En este caso, se generan vehículos, carriles, y semáforos (intersecciones).

In [4]:
# Simulacion

# Camino
road_l = 20 * 1000 # 100 km
speed_limit = 40 / 3.6 # km/h to m/s
road = Road(road_l, speed_limit, 2)

# Semaforo

semaforos = []
for s in range(0, road_l, 1000):
    semaforos.append(Semaforo(s, 'green', 30, 30))

# Vehiculos
    # Sedan - vi = 45 km/h, a = 3.5 m/s^2, space = 5 m
    # Van - vi = 37.5 km/h, a = 2.5 m/s^2, space = 7.5 m
    # Bike - vi = 55 km/h, a = 4 m/s^2, space = 3 m
    # Bus/Truck - vi = 40 km/h, a = 1.75 m/s^2, space = 10 m

sedan1 = Vehicle(45/3.6, 3.5, 0, -1, 'sedan')
sedan2 = Vehicle(45/3.6, 3.5, 10, -1, 'sedan') 
sedan3 = Vehicle(45/3.6, 3.5, 15, -1, 'sedan')
sedan4 = Vehicle(45/3.6, 3.5, 20, -1, 'sedan')
sedan5 = Vehicle(45/3.6, 3.5, 25, -1, 'sedan')
van1 = Vehicle(37.5/3.6, 2.5, 5, -1, 'van')
van2 = Vehicle(37.5/3.6, 2.5, 20, -1, 'van')
van3 = Vehicle(37.5/3.6, 2.5, 25, -1, 'van')
bike1 = Vehicle(55/3.6, 4, 20, -1, 'bike')
bike2 = Vehicle(55/3.6, 4, 15, -1, 'bike')
bike3 = Vehicle(55/3.6, 4, 10, -1, 'bike')
bike4 = Vehicle(55/3.6, 4, 5, -1, 'bike')
bus1 = Vehicle(40/3.6, 1.75, 40, -1, 'bus')
bus2 = Vehicle(40/3.6, 1.75, 30, -1, 'bus')

vehicles = [sedan1, sedan2, sedan3, van1, van2, bike1, bike2, bus1, bus2]

# assign lanes 
def assign_lane(vehicles, road, vehicle):
    position = vehicle.pos
    # Find the available lanes at the current position
    available_lanes = [lane for lane in range(road.lanes) if not any(v.pos == position and v.lane == lane for v in vehicles)]
    
    if not available_lanes:
        # If no lanes are available, move the vehicle ahead 5 meters until assigned
        vehicle.pos += 5
        assign_lane(vehicles, road, vehicle)
    else:
        # Assign the vehicle to the leftmost available lane
        vehicle.lane = available_lanes[0]

for v in vehicles:
    assign_lane(vehicles, road, v)



# Simulacion
duration = 0.5 # horas
dt = 5 # segundos

flow_speeds = simulate_traffic(road, vehicles, semaforos, duration, dt)

##### GIF

Finalmente, se genera un GIF con la simulación del tráfico.

In [10]:
import os

output_directory = "screens4"

# Function to convert flow speeds into visual representation
def draw_flow_speed(flow_speed, index):
    plt.figure(figsize=(10, 4))
    plt.plot([flow_speed[0] / 3600], [flow_speed[1] * 3.6], 'ro', markersize=5)
    plt.xlabel('Time (hours)')
    plt.ylabel('Speed (km/h)')
    plt.title('Average Speed vs. Time')
    plt.grid(True)

    # Save the figure as an image
    image_name = os.path.join(output_directory, f"frame_{index:03d}.png")
    plt.savefig(image_name, format="png")
    plt.close()
    return image_name

# Create frames for the GIF
gif_frames = []
for i, flow_speed in enumerate(flow_speeds):
    frame_image = draw_flow_speed(flow_speed, i)
    gif_frames.append(Image.open(frame_image))

# Save frames as an animated GIF
gif_frames[0].save(
    'traffic_flow.gif',
    save_all=True,
    append_images=gif_frames[1:],
    loop=0,
    duration=1000 // frame_rate
)

# Display the GIF
display(IPImage('traffic_flow.gif'))

-----------

##### **Preguntas**

1. ¿Qué es el modelado multiescala y por qué es valioso para comprender el flujo y la congestión del tráfico urbano?

El modelado multiescala es un enfoque que permite modelar un sistema a diferentes escalas. En el caso del tráfico urbano, se puede modelar el flujo de tráfico a nivel de vehículo, a nivel de segmento de carretera y a nivel de intersección. Este enfoque permite tener en cuenta diferentes factores que afectan el flujo de tráfico, como la densidad del tráfico, los límites de velocidad, la congestión, las interacciones entre vehículos, etc.



2. Describe el modelo individual a nivel de vehículo en esta simulación multiescala. ¿Qué parámetros y factores se consideran a esta escala?

En el caso del vehículo individual, se ignora la necesidad de girar; usamos la suposición de una línea perfectamente recta de vehículos. El modelo a nivel de vehículo tiene en cuenta la velocidad, la aceleración, la posición en la carretera y el carril en el que se encuentra el vehículo. A nivel de funcionamiento, este tiene considerado los parámetros de aceleración y desaceleración, la velocidad máxima, la distancia de seguridad y los altos en las luces de tráfico. De esta manera, se puede modelar el comportamiento de un vehículo individual en una carretera. 



3. Explique el modelo a nivel de segmento de carretera. ¿Cómo tiene en cuenta la densidad del tráfico, los límites de velocidad y la congestión?

La densidad de tráfico, la congestión y los límites de velocidad son tomados en cuenta a la hora de declarar el camino como tal. La modelación toma en cuenta una progresión de los agentes vehículo cada 5 segundos, y al cabo de estos el segmento actual de camino actualiza el estado y las interacciones entre vehículos y semáforos.

Para nivelar la congestión, se distribuyen los vehículos en los carriles despejados, en las posiciones disponibles; de no existir una obvia, se cambia la posición del vehículo. Se ha de dejar en claro que este es el paso inicial, y no refleja el manejo a tiempo real de la congestión: así como en una carretera, mágicamente nadie llegará a distribuir los vehículos en los carriles, en este modelo se hace de manera automática únicamente al principio. En cuanto a los límites de velocidad, los vehículos están programados para desacelerar en relación a su tas de aceleración en función del tipo de vehículo que es. Y el flujo total se considera como la velocidad promedio de los vehículos presentes en la autopista.



4. Analice el modelo a nivel de intersección. ¿Qué variables se tienen en cuenta al simular las interacciones de vehículos en las intersecciones?

Las intersecciones son simuladas utilizando semáforos, los cuales tienen el objetivo de simular los paros en intersecciones, afectando el flujo del segmento de camino en el momento en que los vehículos paran. De esta manera, logramos simular de manera correcta las intersecciones en su impacto con un camino en observación. Como ya se ha mencionado, los vehículos están programados para tener ciertas acciones e interacciones al momento de encontrarse con un semáforo, y estas son las que se tienen en cuenta al simular las interacciones de vehículos en las intersecciones.